In [1]:
import pandas as pd
import os

RAW_DATA_DIR = "../data/01_raw"
YIELD_DATA_PATH = os.path.join(RAW_DATA_DIR, "crop_yield.csv")
SOIL_DATA_PATH = os.path.join(RAW_DATA_DIR, "Soil Data.csv")

PROCESSED_DATA_DIR = "../data/02_intermediate"
MERGED_DATA_PATH = os.path.join(PROCESSED_DATA_DIR, "01_merged_data.csv")

# Ensure the output directory exists
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)

print(f"Loading data from: {YIELD_DATA_PATH} and {SOIL_DATA_PATH}")

# --- 2. Load Datasets ---
df_yield = None
df_soil = None

try:
    df_yield = pd.read_csv(YIELD_DATA_PATH)
    df_soil = pd.read_csv(SOIL_DATA_PATH)
    
    print("Data loaded successfully.")

except FileNotFoundError as e:
    print(f"\n--- FATAL ERROR: File Not Found ---")
    print(f"Error: {e}")
    print("Please ensure your notebook is in the '03_notebooks' folder and the data is in 'data/01_raw'.")
except Exception as e:
    # Catch any other loading errors
    print(f"\n--- FATAL ERROR loading data: {type(e).__name__} - {e}")

# --- 3. Run Processing ONLY if data loaded ---
# This structure replaces the 'exit()' call, which is not suitable for notebooks.

if df_yield is not None and df_soil is not None:
    
    print("\n--- Initial Yield Data Info ---")
    df_yield.info()
    print("\n--- Initial Soil Data Info ---")
    df_soil.info()

    # --- 4. Clean Crop Yield Data (df_yield) ---
    print(f"\n--- Cleaning Crop Yield Data ---")
    
    # Check for missing values
    print(f"Missing values in crop_yield.csv:\n{df_yield.isnull().sum()}")
    initial_rows_yield = len(df_yield)
    df_yield.dropna(subset=['Production', 'Yield'], inplace=True)
    print(f"Dropped {initial_rows_yield - len(df_yield)} rows with missing Production/Yield.")

    # Check for duplicates
    initial_duplicates_yield = df_yield.duplicated().sum()
    if initial_duplicates_yield > 0:
        df_yield.drop_duplicates(inplace=True)
        print(f"Dropped {initial_duplicates_yield} duplicate rows.")
    else:
        print("No duplicate rows found in yield data.")

    # Clean 'State' column
    df_yield['State'] = df_yield['State'].str.strip()


    # --- 5. Clean and Prepare Soil Data (df_soil) ---
    print(f"\n--- Cleaning and Preparing Soil Data ---")
    
    # Rename columns (as we discussed, handles 'STATE' and 'DISTRICT')
    df_soil.rename(columns={
        'STATE': 'State',
        'DISTRICT': 'District',
        'N (in mg/kg)': 'N_SOIL',
        'P (in mg/kg)': 'P_SOIL',
        'K (in mg/kg)': 'K_SOIL',
        'pH': 'pH_SOIL'
    }, inplace=True)
    
    # Check for missing values
    print(f"Missing values in Soil Data.csv:\n{df_soil.isnull().sum()}")

    # Clean 'State' column
    df_soil['State'] = df_soil['State'].str.strip()

    # Aggregate soil data
    print("Aggregating soil data from District to State level (using mean)...")
    soil_cols = ['N_SOIL', 'P_SOIL', 'K_SOIL', 'pH_SOIL']
    df_soil_agg = df_soil.groupby('State')[soil_cols].mean().reset_index()

    print("Aggregated Soil Data (Head):")
    print(df_soil_agg.head())


    # --- 6. Merge Datasets ---
    print(f"\n--- Merging Datasets on 'State' column ---")
    # Use a 'left' merge to keep all records from df_yield
    df_merged = pd.merge(df_yield, df_soil_agg, on='State', how='left')

    print("Merge complete. Info of merged DataFrame:")
    df_merged.info()

    # Handle NaNs from merge
    print(f"\nMissing values post-merge:\n{df_merged.isnull().sum()}")
    for col in soil_cols:
        if df_merged[col].isnull().any():
            median_val = df_merged[col].median()
            df_merged[col].fillna(median_val, inplace=True)
            print(f"Filled NaNs in '{col}' with median value ({median_val}).")

    print(f"\nFinal missing values check:\n{df_merged.isnull().sum()}")

    # --- 7. Save Processed Data ---
    try:
        df_merged.to_csv(MERGED_DATA_PATH, index=False)
        print(f"\nSuccessfully completed Milestone 1.")
        print(f"Cleaned and merged data saved to: {MERGED_DATA_PATH}")
        print("\n--- Final Merged Data (Head) ---")
        print(df_merged.head())
    except IOError as e:
        print(f"Error saving file: {e}")

else:
    print("\n--- Processing skipped due to data loading failure. ---")
    print("Please check the 'FATAL ERROR' message above and correct the file path or issue.")

Loading data from: ../data/01_raw\crop_yield.csv and ../data/01_raw\Soil Data.csv
Data loaded successfully.

--- Initial Yield Data Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19689 entries, 0 to 19688
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Crop             19689 non-null  object 
 1   Crop_Year        19689 non-null  int64  
 2   Season           19689 non-null  object 
 3   State            19689 non-null  object 
 4   Area             19689 non-null  float64
 5   Production       19689 non-null  int64  
 6   Annual_Rainfall  19689 non-null  float64
 7   Fertilizer       19689 non-null  float64
 8   Pesticide        19689 non-null  float64
 9   Yield            19689 non-null  float64
dtypes: float64(5), int64(2), object(3)
memory usage: 1.5+ MB

--- Initial Soil Data Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 786 entries, 0 to 785
Data columns (total 6 columns):
